# Summarizing a news article using Kodexa

In this notebook we're going to walk through the steps to set up a pipeline in Kodexa that will summmarize an online news article.  This example explains the major components of a Kodexa document and attempts to provide a clear, real-world example in using them.

By the end of this example, you'll be able to create a pipeline that reads a page from a website, parses that HTML, identifies/tags data that should be included for summarization, and then uses Kodexa's text summarization action to create a summary of the original content.

Here's the article we'll be using:  [CNN Money News Article](https://money.cnn.com/2018/11/01/retirement/irs-contributions/index.html?utm_source=feedburner&utm_medium=feed&utm_campaign=Feed%3A+rss%2Fmoney_latest+%28CNNMoney%3A+Latest+News%29)


## Setup our imports
1. Our actions will run in the could, we'll need to import the KodexaPlatform and KodexaAction modules
2. In order to build a pipeline in Kodexa, we'll need to import (you guessed it), the Kodexa Pipeline.
3. All files that have been processe become Kodexa Documents, so we'll import that module as well.

We'll be using several different KodexaActions that are executed via the KodexaPlatform, and we'll introduce them as we work through the example.

To interact with the platform, you'll need to register for an account and generate an access token.  If you haven't done that already, follow the steps in our [Getting Started](https://developer.kodexa.com/kodexa-cloud/accessing-kodexa-cloud) guide.



In [1]:
# The kodexa package is public
from kodexa import Document, Pipeline, KodexaAction, KodexaPlatform

# This is the host of our KodexaPlatform & where our KodexaActions will be executed
CLOUD_URL = 'https://platform.kodexa.com'

# The URL for the news story we'll be working with
NEWS_STORY_URL = 'https://money.cnn.com/2018/11/01/retirement/irs-contributions/index.html?utm_source=feedburner&utm_medium=feed&utm_campaign=Feed%3A+rss%2Fmoney_latest+%28CNNMoney%3A+Latest+News%29'


## Set Platform Environment and Access Token Credential

In the next cell, you'll be prompted to enter your access token that you've created in the environment specified by the CLOUD_URL.
If you haven't created a token already, follow the steps in our [Getting Started](https://developer.kodexa.com/org-management/manage-access-token) guide.

* Note:  The text you enter in the prompt field will be masked.  Once you're done entering the access token value, hit enter to complete the action in the cell.

In [2]:
import getpass

ACCESS_TOKEN = getpass.getpass("Enter access token:")

KodexaPlatform.set_url(CLOUD_URL)
KodexaPlatform.set_access_token(ACCESS_TOKEN)

Enter access token: ································


## Get the news story, parse it, and get the Kodexa document

In [3]:

# Create the Pipeline and tell it where to find the source.  
# There are several different ways to specify sources for Pipelines, but when reading from the web, the 'from_url' method is easiest.
pipeline = Pipeline.from_url(NEWS_STORY_URL)

# We know we're reading HTML, so our first step in the Pipeline is to parse it.  
#The parser will produce a Kodexa document, which we'll retrive after the pipeline runs
pipeline.add_step(KodexaAction(slug='kodexa/html-parser', options={"summarize":True,"encoding":"utf8"}, attach_source=True))

# Make it so, Number One
pipeline.run()

kodexa_doc = pipeline.context.output_document



## So what is a Kodexa document, anyway?

Kodexa documents are centered around the content model, and they're designed to allow developers to work with a wide range of unstructured, semi-structured, and structured content. The document has a root content node which has zero-to-many child content nodes. These content nodes are arranged in a hierarchical structure and represent the content of the document.

A content node has several core properties such as type, content, content_parts, children, and feature(s).
* **type:** The specific type of this content node.
* **parent:** Each node is aware of its parent. Only a root content node on a document would not have a parent.
* **features:** Features are a way to store/add additional information about a content node.  The features property on a content node is a collection of features.
* **content:** The text representation of the content.
* **content_parts:** An array version of the content, this is used to break the content and intersperse it with the children, allowing us to understand where the child nodes fit into the content, it is not always present, and is only present if the structure allows for child nodes to be embedded into the content at specific locations.
* **children:** The child content nodes that roll up to this content node.

While at a generic level everything can be thought of as a content node, we leverage the 'type' property on the content node to provide meaning to the hierarchy. In our example, we're using an HTML file, and we should expect to see content nodes with types such as 'div', 'p', 'span', etc.  If we were parsing a PDF file, we may see type values such as 'page' or 'line'.

When source content has been parsed and structured in a Kodexa document, it's broken up into a tree of nodes.  Each of these nodes can capture a portion of the original content (text), and may also have one or more "features". Features allow flexibility in the way that we capture metadata about content, and allow us to add new information to a node that we may want to use in later processing.

For this example, we'll be focusing on the **type**, **features**, and **content** of our content nodes.

More info on Kodexa's Content Model and Document structure can be found here:  [Kodexa Developer Guide](https://developer.kodexa.com/developers/what-is-kodexa/document-structure)


## What types of content nodes are in our document?

To find all content nodes in a document, regardless of type, we'll use the "select" function.  This function is available on both the document and on all of the document's content nodes, so this function can be called on any one of them at any level within the tree structure.  Since we want to find all content nodes starting with the top-most node, we're performing the select at the document level.

## Let's talk about selectors!
When we call a content node's "select" method, we provide a string "selector".  Selectors are derivative of an XPath, so it's similar to how you might use XPath to select content from an XML document.  
 
More information on the syntax of selectors can be found here: [Selector Syntax](https://developer.kodexa.com/developers/documentation/selectors)


In [4]:

# the selector value will return any type of node, with any content
nodes = kodexa_doc.select('//*')

for n in nodes:
    print(f'Type: {n.get_type()}\nContent: {n.get_content()}\n***')


Type: container
Content: None
***
Type: div
Content:    
***
Type: div
Content:    
                               
***
Type: p
Content: None
***
Type: h2
Content: Good news retirement savers: The Internal Revenue Service announced cost of living increases to the contribution limits for retirement-related plans in 2019.
***
Type: p
Content:  Annual contribution limits to 401(k)s will increase to $19,000 from $18,500. 
***
Type: p
Content:  And the annual contribution to an IRA, last increased in 2013, rises to $6,000 from $5,500. 
***
Type: p
Content:  "This is another win for investors and savers," says Stephanie Bacak, a financial planner at Capstone Global Advisors. "For so long there were really no cost of living increases in the IRA so it is a great opportunity for so many to be more prepared for retirement." 
***
Type: p
Content:  Catch-up contributions, available to those age 50 and over, will remain unchanged at $6,000 for 401(k)s and $1,000 for IRAs. 
***
Type: p
Content: None

## What's going on here?  

A Kodexa document consists of multiple ContentNodes, each with a type.  The topmost ContentNode is the "root_node".  You can access it by calling the document's get_root() function.  When we execute select('//*') on the document with that wildcard value, all nodes are included in the resulting list, including the root_node.  That's why we see it in the returned values.  Note - a content node may not have content of its own, but its child nodes may contain content.  It's fine that the get_content() function returns a None value on content nodes.

From the remaining results, we see that there are a couple of divs that have no text content (probably wrapper divs for HTML layout presentation), a couple of paragraph ('p') nodes that have a None value as content, an h2 tag containing a summary of the article, and two spans with date and publisher info.
* The 2 div tags have no content, so we can ignore/omit them.
* The h2 tag is already a summary, but we want to create our own, so we're going to omit it from our summarization logic.
* The span tags containing publisher/date info also won't help our summary, so we'll omit that, too.

It looks like all the text we care about is in in nodes of type 'p' (paragraphs); however a few of them have content values that are empty or None.  We need to omit the nodes with None content values, so we'll filter those out by adding a new tag for only those we want to keep.  

Before we go any further, let's take a look at the node that appears to be blank.  Because this is an HTML document and we're working in Jupyter notebooks in a web browser, HTML non-breaking spaces (&nbsp) render as blank/empty values.  Let's make sure that's not what's going on here.


In [5]:

# We had a node of type 'p' with a blank value - let's take another look
# Note:  the //p selector will return ONLY nodes that are of type 'p'
nodes = kodexa_doc.select('//p')

print(f'\nThe last node is just whitespace? {len(nodes[-1].get_content().strip()) == 0}\n')
    


The last node is just whitespace? True



### That last 'p' just has whitespace.  We're good-to-go!

## How do we identify the data we want to work with?

In Kodexa, we identify data we want to use for downstream processing and then add "tags" to those nodes.  Tagging is a way to provide meaning to a piece of content; for example, you may choose to tag data that repeats at the top of each page as a page header, or you may be working with a spreadsheet and choose to use Kodexa's Excel Table Tagger to tag your data as a table.  A tag is a special type of feature, and is stored just like any other feature.  In our case, we're going to tag the nodes that we'll want to include in our summarization action.  


## Let's tag the nodes that contain data we do want to work with.

Start by creating a new pipeline.  This time we can create our pipeline with the existing kodexa_document, so we don't need to download and parse the HTML.  The "meat" of this pipeline is handeled by the NodeTagger.  Let's dig into this action and see what's going on.

In Kodexa, adding a tag to a node is done using the pattern-based Node Tagger.  This action accepts the following parameters:

* **selector:**  This is a selector value that matches the properties of the node node you’d like to tag.  For example, if you had parsed an HTML document, you may want to tag all paragraphs, divs, and list items - that selector value would be "//p | //div | //li".
* **tag_to_apply:**  This is the tag name you’d like added to all matching nodes.  So, if you want to add the tag “weather”, just pass in the string “weather”.
* **content_re:**  This regular expression value (regex) is used to identify a sub-portion of content in the node(s) identified by the selector.  For example, if you've selected a 'p' node that has the content "The sun is shining brighly today", and you'd like to tag only the words "sun" and "shining", you would provide a content_re value of "(sun|shining)".  A tag would be added to each match in the matching group.
* **use_all_content:**  Boolean parameter indicating if the content_re regex value should be applied only to this node or if it should be applied to this node’s child nodes, as well.  If this parameter is False (the default), only the content value of this node will be used for the content_re match.  If this parameter is True, then this node’s content value and this node’s child nodes content values will be matched to the content_re regex. 
* **node_only:**  Boolean parameter (default False) indicating if the tag_name should be applied at the node level, or if it should be applied more granularly to the matching groups (matching the content_re regex) at their matching indexes.  This parameter is important if you've provided a value for the content_re parameter and there's a match within the node.  If you've set this value to True, then the node will be tagged with the value of the tag_name.  If you've left this False, then the node will be tagged with the tag_name value, but the specific location of the match within the node's content will also be captured.  This allows you to later go back and identify/extract that exact string of text that matched the content_re regex.

Now we can determine the parameter values necessary for tagging our data:
* selector:  We know that the content nodes we want to use for our summary are all of type 'p' and should have a value (must filter out anything empty or None), so we'll provide values for our selector that will limit results to nodes that meet that criteria.
* tag_name: Tag any node that is both of type 'p' (type_re) and has at least one non-whitespace character (content_re) with "include_in_summary".

In [6]:

# Breaking down the values in the selector:
# select all nodes of the document.  Since we perform the select on the document, the select will be run against the root_node and all of its children
selector_val = '//p'

# limit the nodes to only those that have at least one non-whitespace character
selector_val += '[contentRegex("^(.*\S.*)$")]'

# Creating a new pipeline.  This time we don't need to download and parse the HTML, so we'll be creating a pipeline from the existing document
pipeline = Pipeline(kodexa_doc)

# We only want those nodes of type 'p' with at least one non-whitespace character.  Tag any nodes that match with 'include_in_summary'
pipeline.add_step(KodexaAction(slug='kodexa/node-tagger', 
                                     options={'selector':selector_val, 'tag_to_apply':'include_in_summary', 'node_only':True}))

# Engage
pipeline.run()

# Get that freshly tagged document
kodexa_doc = pipeline.context.output_document


## Let's take a look and see what happened

In our node tagging step, our goal was to tag all non-empty and non-None paragraphs with 'include_in_summary'.  Let's see how we made out.

In [7]:

# We'll get all content nodes that have the tag 'include_in_summary', and iterate over them
nodes = kodexa_doc.select("//*[tagRegex('include_in_summary')]")

for n in nodes:
    print(f'Type: {n.get_type()}\nContent: {n.get_content()}\n***')

Type: p
Content:  Annual contribution limits to 401(k)s will increase to $19,000 from $18,500. 
***
Type: p
Content:  And the annual contribution to an IRA, last increased in 2013, rises to $6,000 from $5,500. 
***
Type: p
Content:  "This is another win for investors and savers," says Stephanie Bacak, a financial planner at Capstone Global Advisors. "For so long there were really no cost of living increases in the IRA so it is a great opportunity for so many to be more prepared for retirement." 
***
Type: p
Content:  Catch-up contributions, available to those age 50 and over, will remain unchanged at $6,000 for 401(k)s and $1,000 for IRAs. 
***
Type: p
Content:  In addition to 401(k)s, limits for 403(b)s, most 457 plans and the federal government's Thrift Savings Plan will also increase to $19,000. 
***
Type: p
Content:  Also rising next year are the income ranges that determine eligibility for deductible contributions to IRAs, to Roth IRAs, and to claim the saver's credit. 
***
Type: 

## Hot dog!  Success!

I mean, of course we were going to be successful ;-).  We can see that all the paragraphs for the article have been tagged with the "include_in_summary" tag.  Content nodes of type 'div', 'h2', and 'span' were not tagged, nor were the empty/None 'p' nodes.

## Moving on - extracting sentences

So we've isolated the text we want to include in our summarization.  Now we'll break those paragraphs out into sentences so we can create a summary with them.

To create sentences from the content of each node, we'll use the 'content-nodes-to-sentences-transformer' Kodexa cloud action.  Since we only want to perform the action on the nodes we previously tagged, we'll provide a selector value that limits nodes to only those with the 'include_in_summary' tag.

Again, for the purposes of this example, we're going to run the pipeline with our new step and inspect the results.  In "real life", you'll be able to put all these steps the same pipeline and run them at once.  


In [8]:

pipeline = Pipeline(kodexa_doc) #reusing our document again
pipeline.add_step(KodexaAction(slug='kodexa/content-nodes-to-sentences-transformer', options={'selector':"//*[hasTag('include_in_summary')]"}))

pipeline.run()
kodexa_doc = pipeline.context.output_document


## Inspecting the results

Once again, we'll take a look at the 'p' nodes in our document and see how the previous processing affected the results.

In [9]:

# Selecting all nodes of type 'p'
nodes = kodexa_doc.select('//p')

for n in nodes:
    print(f'{n.get_type()} : {n.get_content()} : {n.get_tags()}')

p : None : []
p :  : ['include_in_summary']
p :  : ['include_in_summary']
p :  : ['include_in_summary']
p :  : ['include_in_summary']
p : None : []
p :  : ['include_in_summary']
p :  : ['include_in_summary']
p :  : ['include_in_summary']
p :  : ['include_in_summary']
p :  : ['include_in_summary']
p :  : ['include_in_summary']
p : None : []
p :       : []


##  Hmm...what's going on here? 

We still have our nodes of type 'p' tagged with 'include_in_summary', but now there's no content.  How is that possible?

Well, let's start at the root_node and get all the content.  The get_all_content() function returns all the content value for the node on which it's called, as well as all of its child nodes.  We're calling get_all_content() from the root_node and, although our root_node doesn't have a content value, its child nodes do.  The get_all_content() call will return all of that in one concatenated string.

In [10]:
kodexa_doc.get_root().get_all_content()

'Good news retirement savers: The Internal Revenue Service announced cost of living increases to the contribution limits for retirement-related plans in 2019. Annual contribution limits to 401(k)s will increase to $19,000 from $18,500. And the annual contribution to an IRA, last increased in 2013, rises to $6,000 from $5,500. "This is another win for investors and savers," says Stephanie Bacak, a financial planner at Capstone Global Advisors. "For so long there were really no cost of living increases in the IRA so it is a great opportunity for so many to be more prepared for retirement." Catch-up contributions, available to those age 50 and over, will remain unchanged at $6,000 for 401(k)s and $1,000 for IRAs.  In addition to 401(k)s, limits for 403(b)s, most 457 plans and the federal government\'s Thrift Savings Plan will also increase to $19,000. Also rising next year are the income ranges that determine eligibility for deductible contributions to IRAs, to Roth IRAs, and to claim the

## We still have content!  Where has it gone?

The content is still on nodes within the document, but is no longer on the nodes of type 'p'.  The 'content-nodes-to-sentences-transformer' step is a transformer which extracts the content for each of the selected nodes, splits that content up into sentences, and creates new nodes of type 'sentence' for each of those sentences.  These new sentence nodes are added as children to the node from which their content was originally extracted.  Finally, the content for those original parent nodes is set to an empty string.

Let's take a look at those sentence nodes!


In [11]:
sentences = kodexa_doc.select("//sentence")

for s in sentences:
    print(f'{s.get_type()} : {s.get_content()}')

sentence : Annual contribution limits to 401(k)s will increase to $19,000 from $18,500.
sentence : And the annual contribution to an IRA, last increased in 2013, rises to $6,000 from $5,500.
sentence : "This is another win for investors and savers," says Stephanie Bacak, a financial planner at Capstone Global Advisors.
sentence : "For so long there were really no cost of living increases in the IRA so it is a great opportunity for so many to be more prepared for retirement."
sentence : Catch-up contributions, available to those age 50 and over, will remain unchanged at $6,000 for 401(k)s and $1,000 for IRAs.
sentence : In addition to 401(k)s, limits for 403(b)s, most 457 plans and the federal government's Thrift Savings Plan will also increase to $19,000.
sentence : Also rising next year are the income ranges that determine eligibility for deductible contributions to IRAs, to Roth IRAs, and to claim the saver's credit.
sentence : For example, the income phase-out range for taxpayers ma

## Yup, our content is still there!  Now let's rank the sentences!

The text has been broken up into individual sentences, each located on their own 'sentence' node.  We can now provide this document with processed sentence nodes to our text summarization functionality.

To extract those sentences that appear to be most significant in the document, we'll be using Kodexa's Significant Text Tagger (TFIDF Implementation).  This action uses a term frequency/inverse document frequency algorithm to rank the significance of the nodes identified by its selector parameter.  Since our selector limits the nodes to our sentences, we are ranking the sentences in the document. 

In [12]:

pipeline = Pipeline(kodexa_doc) #reusing our document again

# selecting nodes of type sentence
pipeline.add_step(KodexaAction(slug='kodexa/significant-tfidf-tagger', 
                                     options={'selector':'//sentence', 'tag_to_apply':'significant_tfidf', 'max_num_results':3}))

pipeline.run()
kodexa_doc = pipeline.context.output_document


## Inspect the results

In [13]:

# selecting all nodes of type 'sentence' that also have the tag 'significant_tfidf'
summaries = kodexa_doc.get_root().select('//sentence[hasTag("significant_tfidf")]')

for s in summaries:
    print(f'{s.get_type()} : {s.get_content()} : {s.get_tags()}')

sentence : Also rising next year are the income ranges that determine eligibility for deductible contributions to IRAs, to Roth IRAs, and to claim the saver's credit. : ['significant_tfidf']
sentence : For example, the income phase-out range for taxpayers making contributions to a Roth IRA increased to $122,000 to $137,000 for singles and heads of household, up from $120,000 to $135,000. : ['significant_tfidf']
sentence : Those that are paid semi-monthly (twice a month or 24 times a year) should be contributing $792 per paycheck and those paid biweekly (every two weeks or 26 times a year) should be contributing $731 per paycheck. : ['significant_tfidf']


## What does this mean?

We see that we have 3 sentences returned - these are the sentences that the action determined to be the most 'relevant' across all the sentences that were selected.  

If we look a little deeper, we can see the ranking order for each of the sentences in our resluts.  The order has been set as an additional feature on each of the significant nodes.  You can access these feature values with the 'get_feature_value' function.  The first parameter matches the 'tag_to_apply' value we passed to the 'significant-tfidf-tagger' action.  The second parameter is the specific feature with the value we'd like to view.

In [14]:

# view rank order of each sentence
for s in summaries:
    print(f"Order: {s.get_feature_value('significant_tfidf', 'order')} :: Content: {s.get_content()}\n")
    

Order: 2 :: Content: Also rising next year are the income ranges that determine eligibility for deductible contributions to IRAs, to Roth IRAs, and to claim the saver's credit.

Order: 3 :: Content: For example, the income phase-out range for taxpayers making contributions to a Roth IRA increased to $122,000 to $137,000 for singles and heads of household, up from $120,000 to $135,000.

Order: 1 :: Content: Those that are paid semi-monthly (twice a month or 24 times a year) should be contributing $792 per paycheck and those paid biweekly (every two weeks or 26 times a year) should be contributing $731 per paycheck.



In [20]:

# Sorting the summaries on 'order' and joining them into on string
sorted_summaries = sorted(summaries, key=lambda s: s.get_feature_value('significant_tfidf', 'order')) 

print(f'\n{" ".join([s.get_content() for s in sorted_summaries])}\n')



Those that are paid semi-monthly (twice a month or 24 times a year) should be contributing $792 per paycheck and those paid biweekly (every two weeks or 26 times a year) should be contributing $731 per paycheck. Also rising next year are the income ranges that determine eligibility for deductible contributions to IRAs, to Roth IRAs, and to claim the saver's credit. For example, the income phase-out range for taxpayers making contributions to a Roth IRA increased to $122,000 to $137,000 for singles and heads of household, up from $120,000 to $135,000.

